In [2]:
# basic imports
import pandas as pd
import sys
import numpy as np
import pandas as pd
import scanpy as sc

# add `Tangram` to path
import tangram as tg
import os

In [3]:
## setting for STARmap
sc_file_path = "/home/comp/cszrwang/data/Breast/spadecon/gm_input_sc.h5ad"
spatial_file_path = "/home/comp/cszrwang/data/Breast/spadecon/gm_input_st.h5ad"
celltype_key = 'celltype'
output_file_path = "/home/comp/cszrwang/results/Deconvolution/results/Breast_1/Tangram/"
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)

In [4]:
ad_sc = sc.read_h5ad(sc_file_path)
ad_sp = sc.read_h5ad(spatial_file_path)

# use raw count both of scrna and spatial
sc.pp.normalize_total(ad_sc)
celltype_counts = ad_sc.obs[celltype_key].value_counts()
celltype_drop = celltype_counts.index[celltype_counts < 2]
print(f'Drop celltype {list(celltype_drop)} contain less 2 sample')
ad_sc = ad_sc[~ad_sc.obs[celltype_key].isin(celltype_drop),].copy()
sc.tl.rank_genes_groups(ad_sc, groupby=celltype_key, use_raw=False)
markers_df = pd.DataFrame(ad_sc.uns["rank_genes_groups"]["names"]).iloc[0:200, :]
print(markers_df)
genes_sc = np.unique(markers_df.melt().value.values)
print(genes_sc)
genes_st = ad_sp.var_names.values
genes = list(set(genes_sc).intersection(set(genes_st)))

Drop celltype [] contain less 2 sample
      B-cells    CAFs Cancer Epithelial Endothelial   Myeloid  \
0        CD74  LGALS1            NDUFB9      IFITM3      CD74   
1     HLA-DRA     BGN             MIEN1       NPDC1  HLA-DPB1   
2    HLA-DPA1    CD63             EIF3H       CRIP2   HLA-DRA   
3        CD37    PPIB            SPINT2       RAMP2  HLA-DPA1   
4       MS4A1     DCN            MRPL13      IGFBP7    TYROBP   
..        ...     ...               ...         ...       ...   
195     RPS12  DPYSL3              RBX1       NUCB1      NAIP   
196     PLCG2   SULF1              MSL1       PREX2  HLA-DQB2   
197   RNASET2   SERF2             CPNE3      NOTCH4    SLC8A1   
198      PKIG     GEM              RPS5        MYH9    CCL4L2   
199    RPL37A    FRZB             FOXA1        SELE    ABHD12   

    Normal Epithelial       PVL T-cells  
0                PERP    IGFBP7    SRGN  
1                RPL8     CALD1    CD3E  
2             TACSTD2    IFITM3   RPS27  
3           

In [5]:
tg.pp_adatas(ad_sc, ad_sp, genes=genes)

ad_map = tg.map_cells_to_space(
                   ad_sc,
                   ad_sp,
                   mode='clusters',
                   cluster_label=celltype_key)

# ad_map = tg.map_cells_to_space(
#                    ad_sc,
#                    ad_sp,
#                    mode="constrained",
#                    target_count=ad_sp.obs.cell_count.sum(),
#                    density_prior=np.array(ad_sp.obs.cell_count) / ad_sp.obs.cell_count.sum(),
#                    num_epochs=1000,
#                    cluster_label=celltype_key)

tg.project_cell_annotations(ad_map, ad_sp, annotation=celltype_key)

celltype_density = ad_sp.obsm['tangram_ct_pred']
celltype_density = (celltype_density.T/celltype_density.sum(axis=1)).T

celltype_density.to_csv(output_file_path + '/Tangram_result.csv')

INFO:root:1290 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:17172 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 1290 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.450, KL reg: 0.403
Score: 0.703, KL reg: 0.001
Score: 0.703, KL reg: 0.001
Score: 0.704, KL reg: 0.001
Score: 0.704, KL reg: 0.001
Score: 0.704, KL reg: 0.001
Score: 0.704, KL reg: 0.001
Score: 0.704, KL reg: 0.001
Score: 0.704, KL reg: 0.001
Score: 0.704, KL reg: 0.001


INFO:root:Saving results..
INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.
